# Example processing script for SEVIRI data
### A more complex example than for SLSTR, using a land-mask and ERA5 wind profiles.

This script requires MSG/SEVIRI data in native format (although it can be easily edited to use HRIT data instead).

Data is available for download at https://data.eumetsat.int

ERA5 data is included in the Examples directory. These files were generated using Copernicus Climate Change Service information (2022).

In this example we compute the sea surface reflectance and also the BRDF values:

    - rho_0d: Solar beam to satellite view reflectances
    - rho_0d: Solar beam to diffuse reflectances
    - rho_dv: Diffuse to satellite view reflectances
    - rho_dd: Diffuse to diffuse reflectances
    
We also apply a land/sea mask (derived from the NASA SRTM data and remapped onto the SEVIRI grid) to mask out all non-water pixels.

In [2]:
%load_ext autoreload
%autoreload 2

# If you have a powerful machine, the dask library can actually slow things down.
# These lines limit the resources dask uses, you may need to experiment with the
# `num_workers` argument to find what works best on your own machine.
import dask
dask.config.set({'array.chunk-size': '32M', 'num_workers': 4})

from dask.diagnostics import ResourceProfiler, Profiler, CacheProfiler, visualize

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


C:\Users\Simon\miniconda3\lib\site-packages\distributed\node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 55556 instead
  warnings.warn(


In [3]:
from satpy import Scene, find_files_and_readers
from pyresample import create_area_def
from datetime import datetime
from netCDF4 import Dataset
import dask.array as da
import xarray as xr
import numpy as np
import pycoxmunk

import warnings
warnings.filterwarnings('ignore')

In [4]:
# Filename of the input granule
sev_fname = 'd:/sat_data/SEV/MSG3-SEVI-MSG15-0100-NA-20160326144242.057000000Z-NA.nat'

# Landmask filename
lsm_fname = './Examples/SEVIRI_LANDMASK.tiff'

# Filenames of the U and V 10m wind components, here taken from ERA5
u10_fname = './Examples/ecmwf-era5_oper_an_sfc_201603261500.10u.nc'
v10_fname = './Examples/ecmwf-era5_oper_an_sfc_201603261500.10v.nc'

# Band name to load / process
bnames = ['VIS006']#, 'VIS008', 'IR_016']

# Cache dir for use in satpy's resample routines
# This is optional, but can help speed things up - especially for 
# geostationary satellites where the area covered is a constant region.
cache_dir = 'd:/sat_data/SEV/cache/'

In [5]:
# A helper function needed for ECMWF processing
def load_wind(fname, var, ecm_scn=None, ftype='nc', dater=datetime(2000, 1, 1, 0, 0, 0)):
    """Load wind datasets from a file.
    
    This assumes that the winds are in individual netCDF files. It also makes a lot of
    assumptions about the data type and area covered. Thus, it will probably not work
    for anything other than the ERA5 data used for internal testing but it can be the
    basis for your own wind loading functions.
    
    Inputs:
     - fname: String, the input filename.
     - var: String, name of variable to read
     - ecm_scn: Scene, an existing scene to save data into. If None, a new Scene is created.
     - ftype: String, file type. Currently only netCDF ('nc') is supported.
     - dater: DateTime, time to set as Scene start_time.
    Returns:
      - ecm_scn: Scene, containing 'u10' and 'v10' winds.
    """
    if ftype != 'nc':
        raise ValueError("Only netCDF winds are supported at present.")
    
    # Open the netCDF file
    fid = Dataset(fname, 'r')
    # Load the variable and retrieve the lats + lons
    inv = np.array(fid[var]).squeeze()
    lat = np.array(fid['latitude']).squeeze()
    lon = np.array(fid['longitude']).squeeze()
    # ERA5 has lats in range 0 -> 360. Here we switch to -180 -> 180
    if np.nanmax(lon) > 180:
        inv = np.roll(inv, np.round(inv.shape[1]/2).astype(int))
        lon = lon - 180.
    # Done with file, close it
    fid.close()
    
    # Create a Scene if there isn't one
    if ecm_scn is None:
        ecm_scn = Scene()
    
    # Create an area for use in resampling.
    # This assumes lat/lon gridded data
    area_ext = (np.nanmin(lon), np.nanmin(lat), np.nanmax(lon), np.nanmax(lat))
    targ_area = create_area_def("source_area",
                                "EPSG:4326",
                                area_extent=area_ext,
                                width=inv.shape[1],
                                height=inv.shape[0])

    # Create a new dataset in the Scene for the wind data
    ecm_scn[var] = xr.DataArray(da.from_array(inv),
                                coords={'y': lat, 'x': lon},
                                attrs={'start_time': dater})

    ecm_scn[var].attrs['area'] = targ_area
    
    # Return the scene
    return ecm_scn

In [6]:
# Create a scene for the granule and load data
scn = Scene([sev_fname], reader='seviri_l1b_native')
scn.load(bnames, upper_right_corner='NE')

In [7]:
# Load the ECMWF wind
ecm_scn = load_wind(u10_fname, 'u10', ecm_scn=None)
ecm_scn = load_wind(v10_fname, 'v10', ecm_scn=ecm_scn)

In [8]:
# Load the land mask
lsm_scn = Scene([lsm_fname], reader='generic_image')
lsm_scn.load(['image'])

In [9]:
# Resample the ECMWF wind data onto the satellite grid
ecm_scn2 = ecm_scn.resample(scn[bnames[0]].attrs['area'],
                            resampler='nearest', cache_dir=cache_dir,
                            radius_of_influence=150000)

In [10]:
# Copy the winds from the ECMWF scene into the satellite scene.
# This simplifies calling PyCoxMunk, as we only need to pass
# one scene instead of two
scn['u10'] = ecm_scn2['u10'].copy()
scn['v10'] = ecm_scn2['v10'].copy()

In [11]:
# Create the PyCoxMunk class
pcm = pycoxmunk.PyCoxMunk(scn, bnames, angle_names='calc', delete_when_done=False, mask_bad=False, do_brdf=True)

In [12]:
# Set up the wind variables within PyCoxMunk
pcm.setup_wind(scn['u10'], scn['v10'])

In [13]:
# Set up the pixel masking
pcm.setup_pixmask(land_mask=np.array(lsm_scn['image'].data).squeeze())

In [14]:
# Retrieve the sea surface reflectance
pcm.retr_coxmunk_refl()

In [16]:
# Save results and intermediate variables to disk.
pcm.scn.save_dataset('VIS006', base_dir='d:/sat_data/sev/out/', enhance=False, dtype=np.float32)

2022-10-19 14:06:46,688 - distributed.protocol.core - CRITICAL - Failed to Serialize
Traceback (most recent call last):
  File "C:\Users\Simon\miniconda3\lib\site-packages\distributed\protocol\core.py", line 109, in dumps
    frames[0] = msgpack.dumps(msg, default=_encode_default, use_bin_type=True)
  File "C:\Users\Simon\miniconda3\lib\site-packages\msgpack\__init__.py", line 38, in packb
    return Packer(**kwargs).pack(o)
  File "msgpack/_packer.pyx", line 294, in msgpack._cmsgpack.Packer.pack
  File "msgpack/_packer.pyx", line 300, in msgpack._cmsgpack.Packer.pack
  File "msgpack/_packer.pyx", line 297, in msgpack._cmsgpack.Packer.pack
  File "msgpack/_packer.pyx", line 264, in msgpack._cmsgpack.Packer._pack
  File "msgpack/_packer.pyx", line 231, in msgpack._cmsgpack.Packer._pack
  File "msgpack/_packer.pyx", line 231, in msgpack._cmsgpack.Packer._pack
  File "msgpack/_packer.pyx", line 264, in msgpack._cmsgpack.Packer._pack
  File "msgpack/_packer.pyx", line 231, in msgpack._cmsg

CancelledError: ('store-map-52016ac13d71f3bf66124520c4ae003c', 0, 3, 0)